Importing the necessary libraries:

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
import keras
from keras.layers import Dense,Conv2D,Dropout,BatchNormalization,Activation,Flatten,MaxPool2D,Input,LeakyReLU
from keras.models import Sequential
from keras.activations import relu
from keras.optimizers import Adam,RMSprop,SGD
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from sklearn import svm
from keras.preprocessing import image
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os
import cv2
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sk
%matplotlib inline
import h5py
import re

In [105]:
!gdown https://drive.google.com/u/0/uc?id=1LnJyGEmbtcG_WwkEKbVfzRdbMBV0oK6Z&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1LnJyGEmbtcG_WwkEKbVfzRdbMBV0oK6Z
To: /content/blogtext.csv
800MB [00:10, 76.5MB/s]


Importing the data:

In [106]:
text_data=pd.read_csv('/content/blogtext.csv')

Obtaining the first 5 samples of the data:

In [4]:
text_data.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


Obtaining the shape of the data:

In [5]:
text_data.shape

(681284, 7)

Preprocessing and cleaning data:
i)All the non alphabetic characters are removed.
ii)All the letters in the data are set to lower case.
iii)All beginning and ending spaces are removed.

In [107]:
import re
text_data['clean_data']=text_data['text'].apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))

In [108]:
text_data['clean_data']=text_data['clean_data'].apply(lambda x: x.lower())

In [109]:
text_data['clean_data']=text_data['clean_data'].apply(lambda x: x.strip())

In [9]:
text_data.head()

,id,gender,age,topic,sign,date,text,clean_data
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info has been found pages and mb of pdf files ...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,these are the team members drewes van der laag...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks to yahoo s toolbar i can now capture th...


Removing the stopwords in the data:

In [111]:
from nltk.corpus import stopwords

In [110]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [112]:
stopwords=set(stopwords.words('english'))

Filtering words on the basis of whether the words are stopwords or not and joining words that are not stopwords together:

In [113]:
text_data['clean_data']=text_data['clean_data'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))

In [ ]:
text_data.head()

,id,gender,age,topic,sign,date,text,clean_data
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...


Converting data present in the age column to string type:

In [115]:
text_data['age']=text_data['age'].astype(str)

In [12]:
text_data['labels']=text_data.apply(lambda col: [col['gender'],(col['age']),col['topic'],col['sign']], axis=1)

KeyboardInterrupt: ignored

Joining labelled data in one column:

In [116]:
text_data['labels']=text_data[['gender','age','topic','sign']].apply(lambda x:','.join(x), axis = 1)

In [13]:
text_data.head()

,id,gender,age,topic,sign,date,text,clean_data,labels
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"male,15,Student,Leo"
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing,"male,15,Student,Leo"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"


In [117]:
text_data1=text_data.drop(columns=['id','date'])

In [15]:
text_data1.head()

,gender,age,topic,sign,text,clean_data,labels
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"male,15,Student,Leo"
1,male,15,Student,Leo,These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,male,15,Student,Leo,testing!!! testing!!!,testing testing,"male,15,Student,Leo"
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius"


In [16]:
text_data1['topic'].unique()

array(['Student', 'InvestmentBanking', 'indUnk', 'Non-Profit', 'Banking',
       'Education', 'Engineering', 'Science', 'Communications-Media',
       'BusinessServices', 'Sports-Recreation', 'Arts', 'Internet',
       'Museums-Libraries', 'Accounting', 'Technology', 'Law',
       'Consulting', 'Automotive', 'Religion', 'Fashion', 'Publishing',
       'Marketing', 'LawEnforcement-Security', 'HumanResources',
       'Telecommunications', 'Military', 'Government', 'Transportation',
       'Architecture', 'Advertising', 'Agriculture', 'Biotech',
       'RealEstate', 'Manufacturing', 'Construction', 'Chemicals',
       'Maritime', 'Tourism', 'Environment'], dtype=object)

Defining the lemmatizer:

In [119]:
from nltk.stem.wordnet import WordNetLemmatizer

In [118]:

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [120]:
text_data1=text_data1[0:5000]

In [121]:
lmtzr = WordNetLemmatizer()
text_data1['lemmatize'] = text_data1['clean_data'].apply(
                    lambda lst:[lmtzr.lemmatize(word) for word in lst.split(" ")])

In [19]:
text_data1

,gender,age,topic,sign,text,clean_data,labels,lemmatize
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"male,15,Student,Leo","[info, found, page, mb, pdf, file, wait, until..."
1,male,15,Student,Leo,These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"male,15,Student,Leo","[team, member, drewes, van, der, laag, urllink..."
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo","[het, kader, van, kernfusie, op, aarde, maak, ..."
3,male,15,Student,Leo,testing!!! testing!!!,testing testing,"male,15,Student,Leo","[testing, testing]"
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius","[thanks, yahoo, toolbar, capture, url, popups,..."
...,...,...,...,...,...,...,...,...
4995,female,17,indUnk,Scorpio,So... I had another one of those dreams...,another one dreams last night except time leas...,"female,17,indUnk,Scorpio","[another, one, dream, last, night, except, tim..."
4996,female,17,indUnk,Scorpio,mmm... strawberry tea for breakfast. To...,mmm strawberry tea breakfast tomorrow think de...,"female,17,indUnk,Scorpio","[mmm, strawberry, tea, breakfast, tomorrow, th..."
4997,female,17,indUnk,Scorpio,"Yay for a new layout!! Yeah, I know, I...",yay new layout yeah know need get complicated ...,"female,17,indUnk,Scorpio","[yay, new, layout, yeah, know, need, get, comp..."
4998,female,17,indUnk,Scorpio,"Ok, so I lied... Fed up isn't playing F...",ok lied fed playing friday night underdog cons...,"female,17,indUnk,Scorpio","[ok, lied, fed, playing, friday, night, underd..."


In [122]:
text_data1['lemmatize1'] = text_data1['lemmatize'].str.join(',')

In [23]:
text_data1

,gender,age,topic,sign,text,clean_data,labels,lemmatize,lemmatize1
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,...",info found pages mb pdf files wait untill team...,"male,15,Student,Leo","[info, found, page, mb, pdf, file, wait, until...","info,found,page,mb,pdf,file,wait,untill,team,l..."
1,male,15,Student,Leo,These are the team members: Drewe...,team members drewes van der laag urllink mail ...,"male,15,Student,Leo","[team, member, drewes, van, der, laag, urllink...","team,member,drewes,van,der,laag,urllink,mail,r..."
2,male,15,Student,Leo,In het kader van kernfusie op aarde...,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo","[het, kader, van, kernfusie, op, aarde, maak, ...","het,kader,van,kernfusie,op,aarde,maak,je,eigen..."
3,male,15,Student,Leo,testing!!! testing!!!,testing testing,"male,15,Student,Leo","[testing, testing]","testing,testing"
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...,thanks yahoo toolbar capture urls popups means...,"male,33,InvestmentBanking,Aquarius","[thanks, yahoo, toolbar, capture, url, popups,...","thanks,yahoo,toolbar,capture,url,popups,mean,s..."
...,...,...,...,...,...,...,...,...,...
4995,female,17,indUnk,Scorpio,So... I had another one of those dreams...,another one dreams last night except time leas...,"female,17,indUnk,Scorpio","[another, one, dream, last, night, except, tim...","another,one,dream,last,night,except,time,least..."
4996,female,17,indUnk,Scorpio,mmm... strawberry tea for breakfast. To...,mmm strawberry tea breakfast tomorrow think de...,"female,17,indUnk,Scorpio","[mmm, strawberry, tea, breakfast, tomorrow, th...","mmm,strawberry,tea,breakfast,tomorrow,think,de..."
4997,female,17,indUnk,Scorpio,"Yay for a new layout!! Yeah, I know, I...",yay new layout yeah know need get complicated ...,"female,17,indUnk,Scorpio","[yay, new, layout, yeah, know, need, get, comp...","yay,new,layout,yeah,know,need,get,complicated,..."
4998,female,17,indUnk,Scorpio,"Ok, so I lied... Fed up isn't playing F...",ok lied fed playing friday night underdog cons...,"female,17,indUnk,Scorpio","[ok, lied, fed, playing, friday, night, underd...","ok,lied,fed,playing,friday,night,underdog,cons..."


Since usage of the entire dataset for training requires high amount of RAM as well as a large amount of time, the number of samples have been drastically reduced in order to suit memory and time requirements.
Setting the input as the lemmatized root tokens and the output as the labels:

In [123]:
data=text_data1[['lemmatize1','labels']]

In [124]:
data1=data[0:5000]

In [125]:
X=data['lemmatize1']

In [126]:
Y=data['labels']

In [127]:
from sklearn.feature_extraction.text import CountVectorizer

Vectorizing the input data using count vectorizer:
The input text data features are using the frequency of occurence of words in each input. 

In [128]:
vectorizer=CountVectorizer(min_df = 2,ngram_range = (1,2),stop_words = "english")

In [129]:
X1=vectorizer.fit_transform(X)

In [ ]:
label_counts=dict()

for labels in data1.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label]+=1
        else:
            label_counts[label]=1

Encoding the labels for multilabel classification:

Converting all labels to lower case and finding the labels from the output.

In [130]:
Y=Y.str.lower()

In [131]:
y = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y]]

Obtaining the labels:

In [132]:
label_counts=dict()

for labels in y:
    for label in labels:
        label=label.lower()
        if label in label_counts:
            label_counts[label]+=1
        else:
            label_counts[label]=1

In [ ]:
label_counts

{'14': 170,
 '15': 339,
 '16': 67,
 '17': 331,
 '23': 137,
 '24': 353,
 '25': 268,
 '26': 96,
 '27': 86,
 '33': 101,
 '34': 540,
 '35': 2307,
 '36': 60,
 '37': 19,
 '39': 79,
 '41': 14,
 '42': 9,
 '44': 3,
 '45': 14,
 '46': 7,
 'accounting': 2,
 'aquarius': 329,
 'aries': 2483,
 'arts': 31,
 'automotive': 14,
 'banking': 16,
 'businessservices': 87,
 'cancer': 94,
 'capricorn': 84,
 'communicationsmedia': 61,
 'consulting': 16,
 'education': 118,
 'engineering': 119,
 'female': 1706,
 'gemini': 86,
 'indunk': 1381,
 'internet': 20,
 'investmentbanking': 70,
 'law': 3,
 'leo': 190,
 'libra': 414,
 'male': 3294,
 'museumslibraries': 2,
 'nonprofit': 47,
 'pisces': 67,
 'religion': 4,
 'sagittarius': 704,
 'science': 33,
 'scorpio': 408,
 'sportsrecreation': 75,
 'student': 569,
 'taurus': 100,
 'technology': 2332,
 'virgo': 41}

In [ ]:
vectorizer_labels = CountVectorizer(min_df = 1,ngram_range = (1,1))
labels_vector = vectorizer_labels.fit_transform(Y)
label_classes=[]
for  key in vectorizer_labels.vocabulary_.keys():
  label_classes.append(key)


In [ ]:
label_classes

['male',
 '15',
 'student',
 'leo',
 '33',
 'investmentbanking',
 'aquarius',
 'female',
 '14',
 'indunk',
 'aries',
 '25',
 'capricorn',
 '17',
 'gemini',
 '23',
 'non',
 'profit',
 'cancer',
 'banking',
 '37',
 'sagittarius',
 '26',
 '24',
 'scorpio',
 '27',
 'education',
 '45',
 'engineering',
 'libra',
 'science',
 '34',
 '41',
 'communications',
 'media',
 'businessservices',
 'sports',
 'recreation',
 'virgo',
 'taurus',
 'arts',
 'pisces',
 '44',
 '16',
 'internet',
 'museums',
 'libraries',
 'accounting',
 '39',
 '35',
 'technology',
 '36',
 'law',
 '46',
 'consulting',
 'automotive',
 '42',
 'religion']

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents='unicode',
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [27]:
X1=vectorizer.transform(X)

In [352]:
X1.shape

(5000, 675910)

Using multilabel binarizer to encode all the labels obtained above:

In [133]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))


In [ ]:
MLB = MultiLabelBinarizer(classes = label_classes)

In [ ]:
Y

0                      male,15,student,leo
1                      male,15,student,leo
2                      male,15,student,leo
3                      male,15,student,leo
4       male,33,investmentbanking,aquarius
                       ...                
4995              female,17,indunk,scorpio
4996              female,17,indunk,scorpio
4997              female,17,indunk,scorpio
4998              female,17,indunk,scorpio
4999              female,17,indunk,scorpio
Name: labels, Length: 5000, dtype: object

In [27]:
Y=Y.str.lower()

In [28]:
y = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y]]

Transforming the inputs to encoded values:

In [134]:
Y1=binarizer.fit_transform(y)

Splitting the data into train and test sets respectively:

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y1,test_size=0.2,random_state=42)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Defining the model:
The logistic regression model is used in order to train the data provided.

In [ ]:
model=LogisticRegression(solver='lbfgs',max_iter = 1000)

In [ ]:
model_rf=RandomForestClassifier()

In [ ]:
model_svm=SVC()

Using one vs rest classifier to implement multilabel classification:

In [ ]:
model=OneVsRestClassifier(model)

In [ ]:
model.fit(Xtrain,Ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

Predicting on the test data using the model that has been trained above:

In [ ]:
y_pred=model.predict(Xtest)

Obtaining the accuracy score:

In [ ]:
accuracy_score(Ytest,y_pred)

0.519

An accuracy of 51.9% is obtained.

Vectorizing the input data using TF-IDF vectorizer and training the model using TF-IDF vectorized data:

In [47]:
X=data['lemmatize1']

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents='unicode',
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [135]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X1,Y1,test_size=0.2,random_state=42)

In [ ]:
model=LogisticRegression(solver='lbfgs',max_iter = 1000)

In [ ]:
model=OneVsRestClassifier(model)

In [ ]:
model.fit(Xtrain,Ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
y_pred=model.predict(Xtest)

In [ ]:
accuracy_score(Ytest,y_pred)

0.377

As seen above, the accuracy score obtained is 37.7% which is lower than that obtained using Count Vectorized data. Thus Count Vectorizer seems to be better in this case.

In [136]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 5.8 MB/s 


In [34]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

Using the binary relevance method where each label is treated as a single class classification problem,i.e the given classification task is broken into n different subtasks,one subtask for each label classification. The classifier used is Gaussian Naive Bayes.

In [ ]:
classifier = BinaryRelevance(GaussianNB())
# train
classifier.fit(Xtrain, Ytrain)

BinaryRelevance(classifier=GaussianNB(priors=None, var_smoothing=1e-09),
                require_dense=[True, True])

In [ ]:
predictions = classifier.predict(Xtest)
# accuracy
print("Accuracy = ",accuracy_score(Ytest,predictions))

Accuracy =  0.218


The accuracy obtained is 21.8% which is poor.

Using label power set:
Here the samples having same labels in the given set of labels are grouped together along with encoded value for each label set.

In [35]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression(max_iter=1000))

In [36]:
classifier.fit(Xtrain, Ytrain)

LabelPowerset(classifier=LogisticRegression(C=1.0, class_weight=None,
                                            dual=False, fit_intercept=True,
                                            intercept_scaling=1, l1_ratio=None,
                                            max_iter=1000, multi_class='auto',
                                            n_jobs=None, penalty='l2',
                                            random_state=None, solver='lbfgs',
                                            tol=0.0001, verbose=0,
                                            warm_start=False),
              require_dense=[True, True])

In [ ]:
predictions = classifier.predict(Xtest)
# accuracy
print("Accuracy = ",accuracy_score(Ytest,predictions))
print("\n")

Accuracy =  0.687




The accuracy is 68.7% which is much better as compared to that of the previous classifiers

Obtaining the classification report:

In [ ]:
y_pred=classifier.predict(Xtest)

In [ ]:
print(classification_report(y_pred,Ytest))

              precision    recall  f1-score   support

           0       0.39      0.67      0.49        21
           1       0.58      0.75      0.65        59
           2       0.07      1.00      0.13         1
           3       0.42      0.66      0.51        41
           4       0.36      0.64      0.46        14
           5       0.51      0.59      0.54        68
           6       0.21      0.54      0.31        24
           7       0.28      0.71      0.40         7
           8       0.50      0.43      0.46        21
           9       0.52      1.00      0.68        13
          10       0.81      0.70      0.75       101
          11       0.96      0.71      0.82       609
          12       0.62      0.80      0.70        10
          13       0.17      1.00      0.29         1
          14       0.43      0.75      0.55         8
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


From the classification report above, it can be seen that the F1 score varies for different classes due to the imbalance in classes as a result of which some classes are found to have relatively high F1 scores while some are found to jave very less score values. The micro average, weighted average as well as samples average scores are found to be high since the take into account the number of samples for each class as well.

Obtaining the index values containing the correctly predicted classes:

In [ ]:
accurate_samples=[]
for i in range(0,y_pred.shape[0]):
  if (y_pred[i]==Ytest[i]).all():
    accurate_samples.append(i)


In [ ]:
len(accurate_samples)

687

Obtaining the predictions for 5 samples in the test dataset that have been correctly predicted:

In [ ]:
for i in accurate_samples[10:15]:
  print("Actual Labels for sample ",i,":",binarizer.inverse_transform(Ytest)[i])
  print("Predicted Labels for sample ",i,":",binarizer.inverse_transform(y_pred)[i])

Actual Labels for sample  14 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  14 : ('35', 'aries', 'male', 'technology')
Actual Labels for sample  15 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  15 : ('35', 'aries', 'male', 'technology')
Actual Labels for sample  16 : ('15', 'female', 'libra', 'student')
Predicted Labels for sample  16 : ('15', 'female', 'libra', 'student')
Actual Labels for sample  17 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  17 : ('35', 'aries', 'male', 'technology')
Actual Labels for sample  18 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  18 : ('35', 'aries', 'male', 'technology')


Using classifier chain:
This is same as that of binary relevance except the consideration of labels classification in a chain as opposed to being considered individually in the case of binary relevancr.

In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression(max_iter=1000))

In [ ]:
classifier.fit(Xtrain, Ytrain)

ClassifierChain(classifier=LogisticRegression(C=1.0, class_weight=None,
                                              dual=False, fit_intercept=True,
                                              intercept_scaling=1,
                                              l1_ratio=None, max_iter=1000,
                                              multi_class='auto', n_jobs=None,
                                              penalty='l2', random_state=None,
                                              solver='lbfgs', tol=0.0001,
                                              verbose=0, warm_start=False),
                order=None, require_dense=[True, True])

In [ ]:
predictions = classifier.predict(Xtest)
# accuracy
print("Accuracy = ",accuracy_score(Ytest,predictions))
print("\n")

Accuracy =  0.584




The accuracy score seen above is 58.4% which is not bad,

In [173]:
y_pred=classifier.predict(Xtest)

In [172]:
from sklearn.metrics import classification_report

In [178]:
Ytest.shape

(1000, 54)

In [179]:
y_pred.shape

(1000, 54)

In [175]:
print(classification_report(y_pred,Ytest))

              precision    recall  f1-score   support

           0       0.39      0.67      0.49        21
           1       0.58      0.75      0.65        59
           2       0.07      1.00      0.13         1
           3       0.42      0.66      0.51        41
           4       0.36      0.64      0.46        14
           5       0.51      0.59      0.54        68
           6       0.21      0.54      0.31        24
           7       0.28      0.71      0.40         7
           8       0.50      0.43      0.46        21
           9       0.52      1.00      0.68        13
          10       0.81      0.70      0.75       101
          11       0.96      0.71      0.82       609
          12       0.62      0.80      0.70        10
          13       0.17      1.00      0.29         1
          14       0.43      0.75      0.55         8
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
y_pred.shape

(1000, 54)

In [61]:
accurate_samples=[]
for i in range(0,y_pred.shape[0]):
  if (y_pred[i]==Ytest[i]).all():
    accurate_samples.append(i)


In [63]:
len(accurate_samples)

687

In [68]:
for i in accurate_samples[10:15]:
  print("Actual Labels for sample ",i,":",binarizer.inverse_transform(Ytest)[i])
  print("Predicted Labels for sample ",i,":",binarizer.inverse_transform(y_pred)[i])

Actual Labels for sample  14 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  14 : ('35', 'aries', 'male', 'technology')
Actual Labels for sample  15 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  15 : ('35', 'aries', 'male', 'technology')
Actual Labels for sample  16 : ('15', 'female', 'libra', 'student')
Predicted Labels for sample  16 : ('15', 'female', 'libra', 'student')
Actual Labels for sample  17 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  17 : ('35', 'aries', 'male', 'technology')
Actual Labels for sample  18 : ('35', 'aries', 'male', 'technology')
Predicted Labels for sample  18 : ('35', 'aries', 'male', 'technology')


Training the model using MLkNN method:

In [137]:
from skmultilearn.adapt import MLkNN

In [151]:
classifier=MLkNN(k=20)
classifier.fit(Xtrain,Ytrain)

MLkNN(ignore_first_neighbours=0, k=20, s=1.0)

In [152]:
accuracy_score(classifier.predict(Xtest),Ytest)

0.38

Training the multilabel classifier using neural network architectures:

i) Defining the input data:

In [104]:
X=text_data1['lemmatize1']

In [109]:
X

0       info,found,page,mb,pdf,file,wait,untill,team,l...
1       team,member,drewes,van,der,laag,urllink,mail,r...
2       het,kader,van,kernfusie,op,aarde,maak,je,eigen...
3                                         testing,testing
4       thanks,yahoo,toolbar,capture,url,popups,mean,s...
                              ...                        
4995    another,one,dream,last,night,except,time,least...
4996    mmm,strawberry,tea,breakfast,tomorrow,think,de...
4997    yay,new,layout,yeah,know,need,get,complicated,...
4998    ok,lied,fed,playing,friday,night,underdog,cons...
4999    well,today,went,church,talked,music,director,t...
Name: lemmatize1, Length: 5000, dtype: object

In [118]:
all_words = ','.join(X).split(',')
all_words

['info',
 'found',
 'page',
 'mb',
 'pdf',
 'file',
 'wait',
 'untill',
 'team',
 'leader',
 'processed',
 'learns',
 'html',
 'team',
 'member',
 'drewes',
 'van',
 'der',
 'laag',
 'urllink',
 'mail',
 'ruiyu',
 'xie',
 'urllink',
 'mail',
 'bryan',
 'aaldering',
 'urllink',
 'mail',
 'het',
 'kader',
 'van',
 'kernfusie',
 'op',
 'aarde',
 'maak',
 'je',
 'eigen',
 'waterstofbom',
 'build',
 'h',
 'bomb',
 'ascott',
 'tartarus',
 'uwa',
 'edu',
 'au',
 'andrew',
 'scott',
 'newsgroups',
 'rec',
 'humor',
 'subject',
 'build',
 'h',
 'bomb',
 'humorous',
 'date',
 'feb',
 'gmt',
 'organization',
 'university',
 'western',
 'australia',
 'original',
 'file',
 'dated',
 'th',
 'november',
 'seemed',
 'transcript',
 'seven',
 'day',
 'article',
 'poorly',
 'formatted',
 'corrupted',
 'added',
 'text',
 'examine',
 'microscope',
 'malleable',
 'like',
 'gold',
 'missing',
 'anyone',
 'full',
 'text',
 'please',
 'distribute',
 'responsible',
 'accuracy',
 'information',
 'converted',
 'h

In [121]:

frequency = pd.Series(all_words).value_counts()[:20]

In [122]:
frequency

like       3266
one        2894
get        2358
know       2213
time       2129
nbsp       1991
go         1985
urllink    1854
think      1816
would      1765
day        1748
thing      1741
really     1721
people     1605
good       1593
well       1451
back       1351
see        1340
got        1340
u          1269
dtype: int64

In [129]:
X2 = X.apply( lambda t : ','.join( word for word in t.split() 
                                                      if word not in freqeuncy) )


In [130]:
X2

0       info,found,page,mb,pdf,file,wait,untill,team,l...
1       team,member,drewes,van,der,laag,urllink,mail,r...
2       het,kader,van,kernfusie,op,aarde,maak,je,eigen...
3                                         testing,testing
4       thanks,yahoo,toolbar,capture,url,popups,mean,s...
                              ...                        
4995    another,one,dream,last,night,except,time,least...
4996    mmm,strawberry,tea,breakfast,tomorrow,think,de...
4997    yay,new,layout,yeah,know,need,get,complicated,...
4998    ok,lied,fed,playing,friday,night,underdog,cons...
4999    well,today,went,church,talked,music,director,t...
Name: lemmatize1, Length: 5000, dtype: object

In [131]:
cv = CountVectorizer(stop_words='english')

In [132]:
X3=cv.fit_transform(X2)

In [133]:
X3.shape

(5000, 29199)

In [27]:
len(X[0])

73

In [36]:
max_len=len(X[0])
for i in X:
  #print(i)
  #break
  if len(i)>max_len:
    max_len=len(i)

In [42]:
max_len

19316

Using tokenizers to generate tokens from the input data:

In [75]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Setting the maximum number of tokens to be generated as 20000:

In [33]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X)

Obtaining the vocabulary size from the tokens obtained above:

In [34]:
vocab_size = len(tokenizer.word_index) + 1

In [40]:
len(tokenizer.word_index.items())

29448

Converting the text present in each input to padded sequences:
The sentences are converted to a sequence of integers for each word present in the sequence.

In [37]:
X2= tokenizer.texts_to_sequences(X)
#X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 19316

X2= pad_sequences(X2, padding='post', maxlen=max_len)

In [86]:
X1= tokenizer.texts_to_matrix(X)

In [141]:
X4.shape

(5000, 19316)

In [103]:
vocab_size

29449

In [32]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2021-07-26 19:37:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-07-26 19:37:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-07-26 19:37:02--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [33]:
!unzip '/content/glove.6B.zip'

Archive:  /content/glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [34]:
from numpy import array
from numpy import asarray
from numpy import zeros

In [35]:
embeddings_dictionary = dict()

glove_file = open('/content/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [44]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
      print('not none')
      embedding_matrix[index] = embedding_vector

Streaming output truncated to the last 5000 lines.
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not none
not 

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.2687    ,  0.81708002,  0.69896001, ..., -0.40110001,
         0.74656999,  0.31121999],
       [-0.22556999,  0.49417999,  0.48609999, ..., -0.45743999,
         0.49645999,  0.34906   ],
       ...,
       [-0.09398   ,  0.64336997, -1.04289997, ...,  0.009265  ,
         1.648     , -0.18978   ],
       [-0.24342   ,  0.91924   , -0.15123001, ...,  0.23467   ,
        -0.14776   , -0.073583  ],
       [-0.36939001, -0.32989001,  1.29820001, ..., -0.52708   ,
         0.74188   ,  0.50866002]])

In [45]:
embedding_matrix.shape

(29449, 100)

In [ ]:
X1.shape

(5000, 19688)

In [46]:
Y1.shape

(5000, 54)

In [43]:
import scipy

In [69]:
X2=X1.copy()

In [34]:
X2=X2.todense()

In [89]:
X2.shape

(5000, 39855)

In [70]:
x_tr,x_test,y_tr,y_test=train_test_split(X2,Y1,test_size=0.2,random_state=42)

In [315]:
X2=X2.sort_indices()

In [354]:
X2

<5000x675910 sparse matrix of type '<class 'numpy.float64'>'
	with 1013810 stored elements in Compressed Sparse Row format>

In [71]:
coo = x_tr.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
x_tr_tensor=tf.SparseTensor(indices, coo.data, coo.shape)

In [73]:
coo = x_test.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
x_test_tensor=tf.SparseTensor(indices, coo.data, coo.shape)

In [360]:
X2_tensor

In [49]:
x_tr,x_test,y_tr,y_test=train_test_split(X2,Y1,test_size=0.2,random_state=42)

In [72]:
X3 = tf.sparse.reorder(x_tr_tensor)

In [74]:
X4 = tf.sparse.reorder(x_test_tensor)

In [40]:
X1

<5000x39855 sparse matrix of type '<class 'numpy.int64'>'
	with 312284 stored elements in Compressed Sparse Row format>

In [ ]:
tokenizer.word_index.items()

dict_items([('like', 1), ('one', 2), ('get', 3), ('know', 4), ('nbsp', 5), ('urllink', 6), ('time', 7), ('would', 8), ('go', 9), ('think', 10), ('really', 11), ('people', 12), ('good', 13), ('well', 14), ('got', 15), ('back', 16), ('see', 17), ('day', 18), ('going', 19), ('much', 20), ('little', 21), ('last', 22), ('even', 23), ('could', 24), ('way', 25), ('want', 26), ('something', 27), ('right', 28), ('today', 29), ('new', 30), ('say', 31), ('make', 32), ('love', 33), ('still', 34), ('went', 35), ('things', 36), ('life', 37), ('said', 38), ('night', 39), ('first', 40), ('work', 41), ('never', 42), ('thing', 43), ('us', 44), ('need', 45), ('take', 46), ('also', 47), ('around', 48), ('feel', 49), ('home', 50), ('two', 51), ('another', 52), ('diva', 53), ('year', 54), ('oh', 55), ('world', 56), ('many', 57), ('thought', 58), ('come', 59), ('always', 60), ('next', 61), ('ever', 62), ('better', 63), ('maybe', 64), ('actually', 65), ('someone', 66), ('years', 67), ('though', 68), ('since',

In [47]:
embedding_matrix.shape

(29449, 100)

In [50]:
X2.shape

(5000, 19316)

In [78]:
X3.shape

TensorShape([4000, 39855])

Using the encoded text sequences to generate embedding vectors of size 300 and passing these vectors to a 1 dimensional Convolution Neural Network:
The loss function used is binary crossentropy while the metric used for measuring accuracy is binary accuracy where each element in the target vector is compared for calculating the accuracy. This metric is considered due to the presence of large number of labels which are in turn used to predict only 4 labels. Thus there is a high ratio of negatives to positives.

In [40]:
model = Sequential()
model.add(Embedding(vocab_size,300, input_length=X2.shape[1]))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=8))
model.add(Flatten())
model.add(Dense(54))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=keras.metrics.BinaryAccuracy())
#     model.add(Dense(32, activation='relu'))
#model.add(Dense(54, activation='softmax'))

In [234]:
history = model.fit(x_tr,y_tr ,batch_size=128, epochs=50, verbose=1,use_multiprocessing=True,validation_data=(x_test,y_test))

Epoch 1/50
32/32 [==============================] - 42s 1s/step - loss: 0.1980 - binary_accuracy: 0.8757 - val_loss: 0.1614 - val_binary_accuracy: 0.9247
Epoch 2/50
32/32 [==============================] - 42s 1s/step - loss: 0.1494 - binary_accuracy: 0.9319 - val_loss: 0.1494 - val_binary_accuracy: 0.9367
Epoch 3/50
32/32 [==============================] - 42s 1s/step - loss: 0.1327 - binary_accuracy: 0.9402 - val_loss: 0.1450 - val_binary_accuracy: 0.9431
Epoch 4/50
32/32 [==============================] - 42s 1s/step - loss: 0.1201 - binary_accuracy: 0.9457 - val_loss: 0.1433 - val_binary_accuracy: 0.9477
Epoch 5/50
32/32 [==============================] - 41s 1s/step - loss: 0.1099 - binary_accuracy: 0.9496 - val_loss: 0.1448 - val_binary_accuracy: 0.9512
Epoch 6/50
32/32 [==============================] - 42s 1s/step - loss: 0.1004 - binary_accuracy: 0.9526 - val_loss: 0.1459 - val_binary_accuracy: 0.9539
Epoch 7/50
32/32 [==============================] - 42s 1s/step - loss: 0.09

In [270]:
y_pred=model.predict(x_test)

In [288]:
y_pred1=y_pred.copy()

In [269]:
y_pred[200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [292]:
accurate_samples=[]
for i in range(0,len(y_pred1)):
  y_pred1[i][y_pred1[i]>=0.5]=1
  y_pred1[i][y_pred1[i]<0.5]=0
  if((np.array(y_pred1[i],dtype='int64')==y_test[i]).all()):
    accurate_samples.append(i)

In [294]:
len(accurate_samples)

350

As seen above,350 samples have been predicted correctly from the test set of 1000 samples.

Using a simple dense neural network:
The activation function used is the sigmoid activation function along with binary crossentropy loss.

Using the count vectorizer matrix of the input:

In [ ]:
X2=X1.copy()

In [ ]:
x_tr,x_test,y_tr,y_test=train_test_split(X2,Y1,test_size=0.2,random_state=42)

Converting the train and test sparse matrices to sparse tensors followed by the reordering of their indices.

In [ ]:
coo = x_tr.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
x_tr_tensor=tf.SparseTensor(indices, coo.data, coo.shape)

In [ ]:
coo = x_test.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
x_test_tensor=tf.SparseTensor(indices, coo.data, coo.shape)

In [ ]:
X3 = tf.sparse.reorder(x_tr_tensor)

In [ ]:
X4 = tf.sparse.reorder(x_test_tensor)

In [78]:
from keras import layers

In [189]:
input_dim = X4.shape[1]

In [190]:
input_dim

19316

In [47]:
X3=tf.sparse.reorder(X3)

In [76]:
X3.shape

TensorShape([4000, 39855])

Defining the neural network:

In [50]:
  # Number of features

model1 = Sequential()
model1.add(layers.Dense(1024, input_dim=X3.shape[1], activation='relu'))
#model1.add(Dropout(0.2))
model1.add(layers.Dense(54, activation='sigmoid'))
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=keras.metrics.BinaryAccuracy())

In [ ]:
history = model1.fit(X3,Y1 ,batch_size=128, epochs=50, verbose=1,use_multiprocessing=True,shuffle=True)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 1024), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_1/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


In [345]:
model1.evaluate(X4,y_test)

32/32 [==============================] - 0s 3ms/step - loss: 0.3401 - binary_accuracy: 0.9968


[0.3401471972465515, 0.9967831969261169]

In [346]:
y_pred=model1.predict(X4)

In [347]:
y_pred1=y_pred.copy()

In [348]:
accurate_samples=[]
for i in range(0,len(y_pred1)):
  y_pred1[i][y_pred1[i]>=0.5]=1
  y_pred1[i][y_pred1[i]<0.5]=0
  if((np.array(y_pred1[i],dtype='int64')==y_test[i]).all()):
    accurate_samples.append(i)

In [349]:
len(accurate_samples)

466

As seen above, 466 samples have been predicted correctly out of 1000 samples.

Adding hidden layers to the neural network along with a dropout of 0.1 between layers:

In [165]:

model1 = Sequential()
model1.add(layers.Dense(1024, input_dim=X3.shape[1], activation='relu'))
model1.add(Dropout(0.1))
model1.add(layers.Dense(512, activation='relu'))
model1.add(Dropout(0.1))
model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dense(128, activation='relu'))
model1.add(layers.Dense(54, activation='sigmoid'))
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=keras.metrics.BinaryAccuracy())

In [166]:
history = model1.fit(X3,y_tr ,batch_size=128, epochs=50, verbose=1,use_multiprocessing=True,shuffle=True)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_14/dense_57/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_14/dense_57/embedding_lookup_sparse/Reshape:0", shape=(None, 1024), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_14/dense_57/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


32/32 [==============================] - 1s 15ms/step - loss: 0.4771 - binary_accuracy: 0.8133
Epoch 2/50
32/32 [==============================] - 0s 15ms/step - loss: 0.1532 - binary_accuracy: 0.9533
Epoch 3/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0954 - binary_accuracy: 0.9709
Epoch 4/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0578 - binary_accuracy: 0.9815
Epoch 5/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0354 - binary_accuracy: 0.9888
Epoch 6/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0198 - binary_accuracy: 0.9939
Epoch 7/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0121 - binary_accuracy: 0.9963
Epoch 8/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0089 - binary_accuracy: 0.9977
Epoch 9/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0062 - binary_accuracy: 0.9985
Epoch 10/50
32/32 [==============================] - 0s 1

In [167]:
model1.evaluate(X4,y_test)

32/32 [==============================] - 0s 3ms/step - loss: 0.2399 - binary_accuracy: 0.9663


[0.23986965417861938, 0.9662591814994812]

In [168]:
y_pred=model1.predict(X4)

In [169]:
y_pred1=y_pred.copy()

In [170]:
accurate_samples=[]
for i in range(0,len(y_pred1)):
  y_pred1[i][y_pred1[i]>=0.5]=1
  y_pred1[i][y_pred1[i]<0.5]=0
  if((np.array(y_pred1[i],dtype='int64')==y_test[i]).all()):
    accurate_samples.append(i)

In [171]:
len(accurate_samples)

556

As seen above,556 samples have been predicted correctly out of 1000 samples which is much higher as compared to the previous outputs.

Thus from the above implementations, it can be seen that dense neural networks that make use of the count vectorized inputs are found to produce much better results as compared to those that are obtained from using the embedding vectors of the tokenized text input sequences that are trained using 1D Convolution Neural Networks. However the best results have been produced by the label powerset multilabel classifier using logistic regression. 

Part 2:

i) Reading the given json file and storing in a corpus variable:




In [1]:
import json

In [2]:
with open('/content/GL Bot.json') as file:
  corpus=json.load(file)

Printing the corpus:

In [4]:
print(corpus)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [183]:
corpus.keys()

dict_keys(['intents'])

In [ ]:
corpus['intents'][0]['tag']

'Intro'

Adding more patterns to intent containing 'SL','NN','Exit' and 'Ticket' tags:





In [80]:
sl_techniques=['linear regression','logistic regression','decision trees','random forest classifier','random forest regressor','ensemble techniques','difference between boosting and bagging','types of nb','types of naive bayes','hyperparameters for different algorithms','loss for different algorithms','best sl algorithm','best supervised learning algorithm','which algorithm gives maximum accuracy','log loss likelihood','types of svm kernels','hyperparameter tuning','grid search','random search','clusters','value of k in knn','packages used for supervised learning','adaboost','ada','boosting','boost','support vectors','support vector machine','svc']

In [6]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append(technique)


In [7]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append('please explain'+' '+technique)

In [8]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append("I don't know" + ' '+technique)

In [9]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append("What is" + ' '+technique)

In [10]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append("What do you mean by" + ' '+technique)

In [11]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append("What is meant by" + ' '+technique)

In [12]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      i['patterns'].append("Can you please tell me" + ' '+technique)

In [81]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for technique in sl_techniques:
      if technique not in i['patterns']:
        i['patterns'].append(technique)


In [13]:
nn_techniques=['cnn','gradient','dropout','dense','sequential','batch normalization','how to use keras','how to use tensorflow','ann vs cnn vs rnn','applications of neural networks','types of nn','types of neural networks','weights','bias','parameters in nn','hyperparameters in nn','transfer learning','epochs','learning rate','optimizer','sgd','adam','adaprop','momentum','nesterov momentum','decay','model checkpoint','steps per epoch','batch size','filters','padding','how to decide padding in cnn','how to decide number of filters in cnn','which is the best transfer learning method','comparison of different architectures','number of layers','neurons to use','neurons','convolution neural networks','residual networks','recurrent neural networks','artificial intelligence','vanishing gradients','exploding gradients','skip connections','activation functions','types of activation functions','lr','cross entropy','binary cross entropy','categorical cross entropy','stochastic gradient','convergence','minima','tanh','softmax','one hot encoding','local minima','unet','rmsprop','convolution','conv2d','conv1d','weights vs bias']

In [14]:
for i in corpus['intents']:
  if i['tag']=='NN':
    for technique in nn_techniques:
      i['patterns'].append(technique)

In [15]:
for i in corpus['intents']:
  if i['tag']=='NN':
    for technique in nn_techniques:
      if technique not in i['patterns']:
        i['patterns'].append(technique)

In [16]:
questions=['what is','what do you mean by','what is meant by','please explain','can you explain','can you tell me',"i'm not able to understand"]

In [17]:
for i in corpus['intents']:
  if i['tag']=='NN':
    for question in questions:
      for technique in nn_techniques:
        i['patterns'].append(question+' '+technique)

In [18]:
exit_words=['good bye','thankyou so much','okay thankyou','you were of great help thanks','thanks','bye bye','tata','see you','okay thanks bye','bye','have a good day','goodnight','my problem has been solved thanks','i have got a solution thankyou','thank you','thanks a lot','okay great thanks','this worked thanks','see you later','bye see you later','thanks for all the help bye','thanks for the support good bye','thanks for the support bye bye','see you soon','appreciate the help bye','appreciate the help good bye','tata bye bye','ta ta','cyu later','this is awesome thanks bye bye','bye take care','take care','this is very nice thanks','will talk to you later','ttyl','gn','thanks gn','omg thanks','omg thank you bye','thanks for your timely help goodbye']

In [19]:
for i in corpus['intents']:
  if i['tag']=='Exit':
    for exit_word in exit_words:
      i['patterns'].append(exit_word)

In [20]:
pm_requests=['could you connect me to the pm','i want to talk to the pm','can i please talk to the pm','i need a better solution','need to raise a request with pm','need to raise request','ticket','connect to pm','need pm to resolve problem','problem not resolved','need more in depth solution','doubt not cleared','need more clarity in the solution','require further clarification']

In [21]:
for i in corpus['intents']:
  if i['tag']=='Ticket':
    for pm_request in pm_requests:
      i['patterns'].append(pm_request)

Adding more responses to the intents containing 'SL' and 'NN' tags:

In [26]:
ml_answers=['please refer to websites and blogs like towardsdatascience,medium.com and official apis for detailed explanation on machine learning related concepts','please go through videos and ppts in olympus for detailed explanation of machine learning concepts','refer to all olympus ppts and videos based on the machine learning topic or discuss with pm']

In [24]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for ml_answer in ml_answers:
      i['responses'].append(ml_answer)

In [27]:
for i in corpus['intents']:
  if i['tag']=='SL':
    for ml_answer in ml_answers:
      if ml_answer not in i['responses']:
        i['responses'].append(ml_answer)

In [28]:
nn_answers=['please refer to websites and blogs like towardsdatascience,medium.com and official apis for detailed explanation on deep learning related concepts','please go through videos and ppts in olympus for detailed explanation of deep learning or neural network concepts','refer to all olympus ppts and videos based on the deep learning topic or discuss with pm']

In [29]:
for i in corpus['intents']:
  if i['tag']=='NN':
    for nn_answer in nn_answers:
      i['responses'].append(nn_answer)

Printing the corpus with the added inputs:

In [30]:
corpus

{'intents': [{'context_set': '',
   'patterns': ['hi',
    'how are you',
    'is anyone there',
    'hello',
    'whats up',
    'hey',
    'yo',
    'listen',
    'please help me',
    'i am learner from',
    'i belong to',
    'aiml batch',
    'aifl batch',
    'i am from',
    'my pm is',
    'blended',
    'online',
    'i am from',
    'hey ya',
    'talking to you for first time'],
   'responses': ['Hello! how can i help you ?'],
   'tag': 'Intro'},
  {'context_set': '',
   'patterns': ['thank you',
    'thanks',
    'cya',
    'see you',
    'later',
    'see you later',
    'goodbye',
    'i am leaving',
    'have a Good day',
    'you helped me',
    'thanks a lot',
    'thanks a ton',
    'you are the best',
    'great help',
    'too good',
    'you are a good learning buddy',
    'good bye',
    'thankyou so much',
    'okay thankyou',
    'you were of great help thanks',
    'thanks',
    'bye bye',
    'tata',
    'see you',
    'okay thanks bye',
    'bye',
    'have 

In [ ]:
type(corpus)

dict

In [31]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Adding all tokens to the W list,tags to the L list and inputs to doc_x list and outputs to doc_y list




In [ ]:
W=[]
L=[]
doc_x=[]
doc_y=[]

In [ ]:
for intent in corpus['intents']:
  for pattern in intent['patterns']:
    w_temp=nltk.word_tokenize(pattern)
    W.extend(w_temp)
    doc_x.append(w_temp)
    doc_y.append(intent['tag'])
  if intent['tag'] not in L:
    L.append(intent['tag'])


In [ ]:
L

['Intro', 'Exit', 'Olympus', 'SL', 'NN', 'Bot', 'Profane', 'Ticket']

In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

In [40]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Implementing lemmatization on the tokens obtained:

In [39]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
W=[lemmatizer.lemmatize(w.lower()) for w in W if w!='?']
W=sorted(list(set(W)))

In [ ]:
W

["'m",
 'a',
 'able',
 'access',
 'accuracy',
 'activation',
 'ada',
 'adam',
 'adaprop',
 'aifl',
 'aiml',
 'algorithm',
 'all',
 'am',
 'an',
 'and',
 'ann',
 'anyone',
 'application',
 'appreciate',
 'architecture',
 'are',
 'artificial',
 'awesome',
 'backward',
 'bad',
 'bagging',
 'batch',
 'bayes',
 'been',
 'belong',
 'best',
 'between',
 'bias',
 'blended',
 'bloody',
 'boosting',
 'bot',
 'buddy',
 'by',
 'bye',
 'can',
 'care',
 'checkpoint',
 'classification',
 'classifier',
 'cluster',
 'cnn',
 'comparison',
 'connection',
 'contact',
 'convolution',
 'create',
 'cross',
 'cya',
 'cyu',
 'day',
 'decay',
 'decide',
 'decision',
 'deep',
 'dense',
 'did',
 'difference',
 'different',
 'diffult',
 'do',
 'dropout',
 'ensemble',
 'epoch',
 'explain',
 'exploding',
 'filter',
 'first',
 'for',
 'forest',
 'forward',
 'from',
 'function',
 'give',
 'gn',
 'good',
 'goodbye',
 'goodnight',
 'got',
 'gradient',
 'great',
 'grid',
 'ha',
 'hate',
 'have',
 'hell',
 'hello',
 'help

In [ ]:
train=[]
target=[]


In [ ]:
out_empty=[0 for i in range(len(L))]

In [ ]:
L

['Intro', 'Exit', 'Olympus', 'SL', 'NN', 'Bot', 'Profane', 'Ticket']

In [ ]:
doc_y

['Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Intro',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Exit',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'Olympus',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'SL',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'NN',
 'Bot',
 'Bot',
 'Bot',
 'Bot',
 'Bot',
 'Bot',
 'Bot',
 'Bot',
 'Profane',
 'Profane',
 'Profane',
 'Profane',
 'Profane',
 'Pro

Creating the bag of words vector:
For each word in the token vector W,1 is added to the bag vector if the word in tokens vector W is found in the words obtained from the input text sentence  and 0 if not found.

In [ ]:
for x,doc in enumerate(doc_x):
  print(x)
  bag=[]
  w_temp=[stemmer.stem(w.lower()) for w in doc]
  for w in W:
    if w in w_temp:
      bag.append(1)
    else:
      bag.append(0)
  output_row=out_empty[:]
  print(doc_y[x])
  output_row[L.index(doc_y[x])]=1
  train.append(bag)
  target.append(output_row)

0
Intro
1
Intro
2
Intro
3
Intro
4
Intro
5
Intro
6
Intro
7
Intro
8
Intro
9
Intro
10
Intro
11
Intro
12
Intro
13
Intro
14
Intro
15
Intro
16
Intro
17
Intro
18
Intro
19
Intro
20
Exit
21
Exit
22
Exit
23
Exit
24
Exit
25
Exit
26
Exit
27
Exit
28
Exit
29
Exit
30
Exit
31
Exit
32
Exit
33
Exit
34
Exit
35
Exit
36
Exit
37
Exit
38
Exit
39
Exit
40
Exit
41
Exit
42
Exit
43
Exit
44
Exit
45
Exit
46
Exit
47
Exit
48
Exit
49
Exit
50
Exit
51
Exit
52
Exit
53
Exit
54
Exit
55
Exit
56
Exit
57
Exit
58
Exit
59
Exit
60
Exit
61
Exit
62
Exit
63
Exit
64
Exit
65
Exit
66
Exit
67
Exit
68
Exit
69
Exit
70
Exit
71
Exit
72
Exit
73
Exit
74
Exit
75
Exit
76
Olympus
77
Olympus
78
Olympus
79
Olympus
80
Olympus
81
Olympus
82
Olympus
83
Olympus
84
Olympus
85
Olympus
86
Olympus
87
Olympus
88
Olympus
89
SL
90
SL
91
SL
92
SL
93
SL
94
SL
95
SL
96
SL
97
SL
98
SL
99
SL
100
SL
101
SL
102
SL
103
SL
104
SL
105
SL
106
SL
107
SL
108
SL
109
SL
110
SL
111
SL
112
SL
113
SL
114
SL
115
SL
116
SL
117
SL
118
SL
119
SL
120
SL
121
SL
122
SL
123
SL
124
S

In [ ]:
target

[[1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 

In [ ]:
np.array(W).shape

(263,)

Defining the dense neural network to train the input bag of words vector and the output ohe hot encoded targets:
The loss used is categorical crossentropy.

In [47]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(target[0]), activation='softmax'))

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(train,target,test_size=0.1,random_state=42)

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
opt=Adam(learning_rate=0.001)

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(x_train), np.array(y_train), epochs=200, batch_size=5, verbose=1,validation_data=(x_test,y_test),shuffle=True)

Epoch 1/200
126/126 [==============================] - 1s 5ms/step - loss: 1.7162 - accuracy: 0.4034 - val_loss: 0.9866 - val_accuracy: 0.5857
Epoch 2/200
126/126 [==============================] - 0s 2ms/step - loss: 1.0744 - accuracy: 0.5721 - val_loss: 0.8010 - val_accuracy: 0.6857
Epoch 3/200
126/126 [==============================] - 0s 2ms/step - loss: 0.8601 - accuracy: 0.6867 - val_loss: 0.6307 - val_accuracy: 0.8286
Epoch 4/200
126/126 [==============================] - 0s 2ms/step - loss: 0.6769 - accuracy: 0.7907 - val_loss: 0.5154 - val_accuracy: 0.8714
Epoch 5/200
126/126 [==============================] - 0s 2ms/step - loss: 0.5202 - accuracy: 0.8325 - val_loss: 0.4700 - val_accuracy: 0.8857
Epoch 6/200
126/126 [==============================] - 0s 2ms/step - loss: 0.4465 - accuracy: 0.8447 - val_loss: 0.4590 - val_accuracy: 0.8714
Epoch 7/200
126/126 [==============================] - 0s 2ms/step - loss: 0.3905 - accuracy: 0.8548 - val_loss: 0.4652 - val_accuracy: 0.8714

In [ ]:
model.save('chatbot2.h5')

In [ ]:
del input

Implementing the chatbot:

In [ ]:
while True:
  bag=[]
  input_model=[]
  inp_sentence=input("\nYou:")
  
  doc=nltk.word_tokenize(inp_sentence)
  w_temp=[lemmatizer.lemmatize(w.lower()) for w in doc]
  for w in W:
    if w in w_temp:
      bag.append(1)
    else:
      bag.append(0)
  
  input_model.append(bag)
  #print(np.array(input_model).shape)
  result=model.predict(np.array(input_model))
  result_index=np.argmax(result)
  tag=L[result_index]
  for tg in corpus['intents']:
    if tag==tg['tag']:
      responses=tg['responses']
  print(random.choice(responses))
  if tag=='Exit':
    break


You:please explain svm
Link: Machine Learning wiki 

You:types of kernels
Link: Machine Learning wiki 

You:svm kernels
Link: Machine Learning wiki 

You:cyu bye
I hope I was able to assist you, Good Bye


Using keras tokenizers followed by the generation of encoded sequences from the input text sentences:

Creating a dataframe of input sentences and corresponding labels

In [82]:
df = pd.DataFrame(columns=['sentences','labels'])

Appending the pattern sentences present in the corpus to the sentences column of the dataframe:

In [83]:
#is_pattern = is_pattern

for intent in corpus['intents']:
    
        for pattern in intent['patterns']:
            w = pattern
            df_to_append = pd.Series([w,intent['tag']], index = df.columns)
            df = df.append(df_to_append,ignore_index=True)

In [84]:
df

,sentences,labels
0,hi,Intro
1,how are you,Intro
2,is anyone there,Intro
3,hello,Intro
4,whats up,Intro
...,...,...
874,problem not resolved,Ticket
875,need more in depth solution,Ticket
876,doubt not cleared,Ticket
877,need more clarity in the solution,Ticket


In [36]:
df.labels.value_counts(sort=False)

Exit        56
Olympus     13
Profane      9
Intro       20
Ticket      23
Bot          8
NN         536
SL         211
Name: labels, dtype: int64

Creating a separate dataframe for responses and the corresponding labels:

In [85]:

df1 = pd.DataFrame(columns=['responses','labels'])

for intent in corpus['intents']:
    for response in intent['responses']:
        w = response
        df_to_append = pd.Series([w,intent['tag']], index = df1.columns)
        df1 = df1.append(df_to_append,ignore_index=True)

In [ ]:
df1

,responses,labels
0,Hello! how can i help you ?,Intro
1,"I hope I was able to assist you, Good Bye",Exit
2,Link: Olympus wiki,Olympus
3,Link: Machine Learning wiki,SL
4,Link: Neural Nets wiki,NN
5,I am your virtual learning assistant,Bot
6,Please use respectful words,Profane
7,Tarnsferring the request to your PM,Ticket


Defining the tokenizer function for generating the tokens,removing punctuations and retaining only words containing alphabets. This is followed by lemmatization to obtain the word roots.

In [86]:
from collections import Counter
import string

In [41]:
lemmatizer = WordNetLemmatizer()

vocab = Counter()
labels = []
def tokenizer(entry):
    tokens = entry.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    tokens = [re_punc.sub('', w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [lemmatizer.lemmatize(w.lower()) for w in tokens]
#     stop_words = set(stopwords.words('english'))
#     tokens = [w for w in tokens if not w in stop_words]
    tokens = [word.lower() for word in tokens if len(word) > 1]
    return tokens

In [42]:
def remove_stop_words(tokenizer,df,feature):
    doc_without_stopwords = []
    for entry in df[feature]:
        tokens = tokenizer(entry)
        #joblib.dump(tokens,'tokens.pkl')
        doc_without_stopwords.append(' '.join(tokens))
        
    df[feature] = doc_without_stopwords
    return

In [43]:
def create_vocab(tokenizer,df,feature):
    for entry in df[feature]:
        tokens = tokenizer(entry)   
        vocab.update(tokens)
    #joblib.dump(vocab,'vocab.pkl')
    return

Calling the functions to bring about the preprocessing:

In [87]:
create_vocab(tokenizer,df,'sentences')
remove_stop_words(tokenizer,df,'sentences')

In [ ]:
df

,sentences,labels,label_enc
0,hi,Intro,2
1,how are you,Intro,2
2,is anyone there,Intro,2
3,hello,Intro,2
4,whats up,Intro,2
...,...,...,...
693,not good solution,Ticket,7
694,no help,Ticket,7
695,wasted my time,Ticket,7
696,useless bot,Ticket,7


In [ ]:
print(vocab.most_common(20))

[('what', 212), ('you', 209), ('is', 155), ('by', 138), ('explain', 119), ('can', 116), ('to', 111), ('please', 93), ('of', 90), ('me', 76), ('do', 70), ('mean', 69), ('tell', 69), ('meant', 69), ('not', 67), ('understand', 64), ('able', 61), ('im', 47), ('learning', 46), ('type', 45)]


In [ ]:
df

,sentences,labels
0,hi,Intro
1,how are you,Intro
2,is anyone there,Intro
3,hello,Intro
4,whats up,Intro
...,...,...
693,not good solution,Ticket
694,no help,Ticket
695,wasted my time,Ticket
696,useless bot,Ticket


In [ ]:
vocab

Counter({'able': 61,
         'access': 1,
         'accuracy': 7,
         'activation': 17,
         'ada': 3,
         'adam': 9,
         'adaprop': 8,
         'aifl': 1,
         'aiml': 1,
         'algorithm': 35,
         'all': 1,
         'am': 17,
         'an': 1,
         'and': 7,
         'ann': 9,
         'anyone': 1,
         'application': 8,
         'appreciate': 2,
         'architecture': 8,
         'are': 9,
         'artificial': 10,
         'awesome': 1,
         'backward': 1,
         'bad': 1,
         'bagging': 8,
         'batch': 18,
         'bayes': 9,
         'been': 1,
         'belong': 1,
         'best': 23,
         'between': 7,
         'bias': 8,
         'blended': 1,
         'bloody': 1,
         'boosting': 13,
         'bot': 2,
         'buddy': 1,
         'by': 138,
         'bye': 18,
         'can': 116,
         'care': 2,
         'checkpoint': 8,
         'classification': 1,
         'classifier': 7,
         'cluster': 7,
 

In [ ]:
Generating the padded encoded sequences:

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Defining the function to create the encoded sequences:

In [46]:
def encoder(df,feature):
#     text = ' '.join(list(vocab.keys()))
    t = Tokenizer()
    entries = [entry for entry in df[feature]]
    t.fit_on_texts(entries)
    #joblib.dump(t,'tokenizer_t.pkl')
    vocab_size = len(t.word_index) + 1
    entries = [entry for entry in df[feature]]
    max_length = max([len(s.split()) for s in entries])
    encoded = t.texts_to_sequences(entries)
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded, vocab_size

Defining the tokenizer:

In [88]:
t = Tokenizer()
entries = [entry for entry in df['sentences']]
t.fit_on_texts(entries)

Creating the encoded sequences:

In [89]:
X,vocab_size = encoder(df,'sentences')

Storing the encoded data in a separate dataframe:

In [90]:

df_encoded = pd.DataFrame(X)

In [91]:
df_encoded['labels'] = df.labels

In [ ]:
df_encoded

,0,1,2,3,4,5,6,7,8,9,10,11,12,labels
0,175,0,0,0,0,0,0,0,0,0,0,0,0,Intro
1,23,63,2,0,0,0,0,0,0,0,0,0,0,Intro
2,3,176,177,0,0,0,0,0,0,0,0,0,0,Intro
3,178,0,0,0,0,0,0,0,0,0,0,0,0,Intro
4,179,180,0,0,0,0,0,0,0,0,0,0,0,Intro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,15,60,137,0,0,0,0,0,0,0,0,0,0,Ticket
694,150,64,0,0,0,0,0,0,0,0,0,0,0,Ticket
695,257,134,155,0,0,0,0,0,0,0,0,0,0,Ticket
696,174,173,0,0,0,0,0,0,0,0,0,0,0,Ticket


Encoding the target variables:

In [92]:

from sklearn.preprocessing import LabelEncoder
lable_enc = LabelEncoder()

In [93]:
df['label_enc'] = lable_enc.fit_transform(df_encoded.labels)

In [ ]:
df

,sentences,labels,label_enc
0,hi,Intro,2
1,how are you,Intro,2
2,is anyone there,Intro,2
3,hello,Intro,2
4,whats up,Intro,2
...,...,...,...
693,not good solution,Ticket,7
694,no help,Ticket,7
695,wasted my time,Ticket,7
696,useless bot,Ticket,7


In [ ]:
df[df['labels']=='SL']

,sentences,labels,label_enc
89,am not able to understand svm,SL,6
90,explain me how machine learning work,SL,6
91,am not able to understand naive bayes,SL,6
92,am not able to understand logistic regression,SL,6
93,am not able to understand ensemble techbniques,SL,6
...,...,...,...
267,what is meant by grid search,SL,6
268,what is meant by random search,SL,6
269,what is meant by cluster,SL,6
270,what is meant by value of in knn,SL,6


In [94]:
df1['label_enc'] = lable_enc.transform(df1.labels)

In [ ]:
df1

,responses,labels,label_enc
0,Hello! how can i help you ?,Intro,2
1,"I hope I was able to assist you, Good Bye",Exit,1
2,Link: Olympus wiki,Olympus,4
3,Link: Machine Learning wiki,SL,6
4,Link: Neural Nets wiki,NN,3
5,I am your virtual learning assistant,Bot,0
6,Please use respectful words,Profane,5
7,Tarnsferring the request to your PM,Ticket,7


Storing the input and outputs in separate variables:

In [95]:
x=df_encoded.drop(columns=['labels'])

In [96]:
y=df_encoded['labels']

In [224]:
df_encoded[df_encoded['labels']=='Bot']

,0,1,2,3,4,5,6,7,8,9,10,11,12,labels
808,1,3,154,187,0,0,0,0,0,0,0,0,0,Bot
809,255,69,2,0,0,0,0,0,0,0,0,0,0,Bot
810,187,8,0,0,0,0,0,0,0,0,0,0,0,Bot
811,256,69,154,149,10,257,0,0,0,0,0,0,0,Bot
812,1,69,154,165,149,0,0,0,0,0,0,0,0,Bot
813,149,10,258,0,0,0,0,0,0,0,0,0,0,Bot
814,165,149,0,0,0,0,0,0,0,0,0,0,0,Bot
815,149,0,0,0,0,0,0,0,0,0,0,0,0,Bot


Binarizing the labels using get_dummies function:

In [97]:
y=pd.get_dummies(y).values

In [223]:
y[673]

array([0, 0, 0, 1, 0, 0, 0, 0], dtype=uint8)

In [225]:
x.shape

(834, 13)

In [226]:
y.shape

(834, 8)

In [ ]:
vocab_size

260

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [98]:
max_length = x.shape[1]

In [99]:
max_length

13

Defining the 1D convolution neural network architecture where the embedding vectors of size 300 are provided as input to the network:

In [100]:
model = Sequential()
model.add(Embedding(vocab_size,300, input_length=max_length))
model.add(Conv1D(filters=64, kernel_size=4, activation='relu'))
model.add(MaxPooling1D(pool_size=8))
model.add(Flatten())
#     model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [101]:
model.compile(loss = 'categorical_crossentropy',
              # optimizer = Adam(lr=0.001),
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:

Splitting the train and test data:

In [102]:
x_tr,x_test,y_tr,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

In [103]:
history = model.fit(x_tr, y_tr, epochs=200, verbose=1,validation_data=(x_test,y_test))

Epoch 1/200
25/25 [==============================] - 1s 12ms/step - loss: 1.4114 - accuracy: 0.5841 - val_loss: 1.0537 - val_accuracy: 0.6705
Epoch 2/200
25/25 [==============================] - 0s 5ms/step - loss: 0.9544 - accuracy: 0.6662 - val_loss: 0.8030 - val_accuracy: 0.7045
Epoch 3/200
25/25 [==============================] - 0s 5ms/step - loss: 0.6662 - accuracy: 0.7927 - val_loss: 0.5539 - val_accuracy: 0.7841
Epoch 4/200
25/25 [==============================] - 0s 5ms/step - loss: 0.4285 - accuracy: 0.8850 - val_loss: 0.3735 - val_accuracy: 0.8864
Epoch 5/200
25/25 [==============================] - 0s 5ms/step - loss: 0.2798 - accuracy: 0.9254 - val_loss: 0.2724 - val_accuracy: 0.9091
Epoch 6/200
25/25 [==============================] - 0s 5ms/step - loss: 0.1899 - accuracy: 0.9532 - val_loss: 0.2256 - val_accuracy: 0.9318
Epoch 7/200
25/25 [==============================] - 0s 5ms/step - loss: 0.1323 - accuracy: 0.9772 - val_loss: 0.1978 - val_accuracy: 0.9432
Epoch 8/200


In [65]:
model.save('chatbot_conv.h5')

In [ ]:
max_length

13

In [72]:
t.word_index

{'able': 17,
 'access': 241,
 'accuracy': 140,
 'activation': 47,
 'ada': 68,
 'adaboost': 151,
 'adam': 81,
 'adaprop': 98,
 'aifl': 212,
 'aiml': 211,
 'algorithm': 25,
 'all': 232,
 'am': 44,
 'an': 262,
 'and': 137,
 'ann': 77,
 'anyone': 202,
 'application': 95,
 'appreciate': 183,
 'architecture': 108,
 'are': 70,
 'artificial': 69,
 'awesome': 235,
 'backward': 261,
 'bad': 278,
 'bagging': 87,
 'batch': 41,
 'bayes': 73,
 'been': 229,
 'belong': 210,
 'best': 35,
 'better': 283,
 'between': 136,
 'bias': 49,
 'binary': 116,
 'blended': 213,
 'bloody': 268,
 'boost': 152,
 'boosting': 38,
 'bot': 194,
 'buddy': 223,
 'by': 4,
 'bye': 43,
 'can': 6,
 'care': 186,
 'categorical': 117,
 'checkpoint': 102,
 'clarification': 292,
 'clarity': 289,
 'classification': 253,
 'classifier': 133,
 'cleared': 288,
 'cluster': 147,
 'cnn': 27,
 'comparison': 107,
 'connect': 197,
 'connection': 114,
 'contact': 245,
 'convergence': 119,
 'convolution': 57,
 'could': 281,
 'create': 280,
 'cro

Creating the chatbot:

In [104]:
while True:
  flag=0
  bag=[]
  doc_without_stopwords = []
  input_model=[]
  inp_sentence=input("\nYou:")
  
  tokens = tokenizer(inp_sentence)
        #joblib.dump(tokens,'tokens.pkl')
  for token in tokens:
    if token not in t.word_index.keys():
      print("I do not understand. Please be more relevant or specific.")
      flag=1
  if flag==1:
    continue
  doc_without_stopwords.append(' '.join(tokens))
  inp_sentence1=doc_without_stopwords
  #t = Tokenizer()
  entries = [entry for entry in inp_sentence1]
  #t.fit_on_texts(entries)
  #joblib.dump(t,'tokenizer_t.pkl')
  #vocab_size = len(t.word_index) + 1
  #entries = [entry for entry in inp_sentence1]
  #max_length = max([len(s.split()) for s in entries])
  encoded = t.texts_to_sequences(entries)
  padded = pad_sequences(encoded, maxlen=max_length, padding='post')
  #input_model.append(bag)
  #print(np.array(input_model).shape)
  result=model.predict(padded)
  #print(result)
  result_index=np.argmax(result)
  #print(result_index)
  tag=lable_enc.inverse_transform([result_index])[0]
  #tag=L[result_index]
  for tg in corpus['intents']:
    if tag==tg['tag']:
      responses=tg['responses']
  print(random.choice(responses))
  if tag=='Exit':
    break


You:support vector
refer to all olympus ppts and videos based on the machine learning topic or discuss with pm

You:support
refer to all olympus ppts and videos based on the machine learning topic or discuss with pm

You:thanks for the support
I hope I was able to assist you, Good Bye
